# Prediction engineering case study using UK Retail Dataset 

In this case study, we will study prediction engineering. Prediction engineering is a step in predictive modeling where we:
* Define an outome we are interested in predicting 
* Scan the data to find the past occurences of the outcome
* Make these past occurences training examples for machine learning/modeling

We will then use [featuretools](featuretools.com) to extract features and learn a predictive model. 

In this particular casestudy, we are focusing a retail dataset openly available at http://archive.ics.uci.edu/ml/datasets/online+retail

We will define the prediction problem as the one where the customer has more than ``k`` purchases

In [1]:
import featuretools as ft
from utils import (find_training_examples, load_uk_retail_data, 
                   engineer_features_uk_retail, preview, feature_importances)
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import Imputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import confusion_matrix
ft.__version__

'0.1.17'

# Step 1:  Load and prepare data 
<p>If you have not yet downloaded the data it can be downloaded <a href="https://s3.amazonaws.com/mit-dsx-data/uk-retail-data.zip">from S3</a>. Once you have downloaded the archive, unzip it and place the uk-retail-data folder in the same directory as this script. 
</p>

In [2]:
item_purchases, invoices, items, customers = load_uk_retail_data()

FileNotFoundError: File b'uk-retail-data/item_purchases.csv' does not exist

The dataset has the following tables:
* ``item_purchases``
* ``invoices``
* ``items``
* ``customers``

The following relations exist
* A customer may have multiple invoices 
* An item may have been purchased multiple times 
* An invoice may have multiple item purchases 

In [ ]:
entities = {
        "item_purchases": (item_purchases, "item_purchase_id", "InvoiceDate" ),
        "items": (items, "StockCode"),
        "customers": (customers,"CustomerID"),
        "invoices":(invoices,"InvoiceNo", "first_item_purchases_time")
        }

relationships = [("customers", "CustomerID","invoices", "CustomerID"), 
                ("invoices", "InvoiceNo","item_purchases", "InvoiceNo"),
                ("items", "StockCode","item_purchases", "StockCode")]

# Step 2 : Find training examples

In the code snippet below, we are trying to find training examples from the data. We set the following parameters:
* ``prediction_window``=14 days 
* ``training_window``=21 days
* ``lead`` = 7 days
* ``threshold``=2 --> specifies the number of purchases that the customer need to have in the future to be considered engaged

In [ ]:
label_times = find_training_examples(item_purchases, invoices,
                                     prediction_window=pd.Timedelta("14d"),
                                     training_window=pd.Timedelta("21d"),
                                     lead=pd.Timedelta("7d"),
                                     threshold=5)

In [ ]:
preview(label_times, 5)

In the output above, we are showing the first 5 training examples. The first column is the CustomerID, the second column is the timestamp after which we can use the data for generating features. The third column is the last timestamp we can use the data from the customer. The fourth column is the label. It is ``True`` if the customer had more than 5 purchases in the period between (``cutoff_time``, ``cutoff_time + prediction_window``)

# Step 3: Now lets generate features. 
Next we generate features for each of the training examples. We use featuretools to generate the features. Featuretools is an automated feature engineering software. We go into detail about this software package in the NYC-Taxi case study. Here we simply use the tool to generate features. 

In [ ]:
feature_matrix = engineer_features_uk_retail(entities, relationships,
                                             label_times, training_window='21d')

In [ ]:
preview(feature_matrix, 10)

# Step 4: Let's train a model using Random Forests 
Now we are ready to train a model and evaluate it. To do this, we:
* First split our training examples in train_test_split 
* Impute missing values 
* Train a model using training data 
* Test on the data set aside for testing

We can split the data using the function ``train_test_split`` and specifying the proportion we want for testing. In this case we specified that as 35%

In [ ]:
label_times[["CustomerID"]]
X_y = feature_matrix.merge(label_times[["CustomerID", 'purchases>threshold']], right_on="CustomerID", left_index=True)
y = X_y.pop('purchases>threshold')
X_train, X_test, y_train, y_test = train_test_split(feature_matrix, 
                                                    y, test_size=0.35)

We can impute the missing values or ``NaN`` values in the feature_matrix using the ``Imputer`` in scikit-learn. It replaces the ``NaN`` values in a feature column with the ``mean`` of the rest of the entries in that column. This is a simple imputation startegy

In [ ]:
imp = Imputer(missing_values='NaN', strategy='mean', axis=0)
imp = imp.fit(X_train)
X_train_imp = imp.transform(X_train)

We can train a RandomForest classifier (a type of ensemble classifier). We make use of scikit-learn package for this as well. 

In [ ]:
clf = RandomForestClassifier(random_state=0, n_estimators=100,
                             class_weight="balanced", verbose=True)
clf.fit(X_train_imp, y_train)

# Step 5: Test the model
To test a model, we:
1. First impute the missing values
* Use the trained classifier to predict the labels

In [ ]:
X_test_imp = imp.transform(X_test)
predicted_labels = clf.predict(X_test_imp)

We evaluate by calculating

In [ ]:
tn, fp, fn, tp = confusion_matrix(y_test, predicted_labels).ravel()

In [ ]:
tn, fp, fn, tp

In [ ]:
feature_importances(clf, feature_matrix.columns, n=15)